# 数据清洗与准备（Data Cleaning and Preparation）

1. 处理缺失值
    1. 过滤缺失值
    2. 补全缺失值
2. 数据转换
    1. 删除重复值
    2. 使用函数或映射进行数据转换
    3. 替代值
    4. 重命名轴索引
    5. 离散化和分箱
    6. 检测和过滤异常值
    7. 置换和随机取样
    8. 计算指标/虚拟变量
3. 字符串操作
    1. 字符串对象方法
    2. 正则表达式
    3. pandas中的向量化字符串函数

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1.处理缺失值（Handling Missing Data）

在pandas中，Python对象的所有描述性统计信息默认情况下是排除缺失值的，它使用浮点值NaN(Not a number)来表示数值型数据的缺失值。

<font color='red'>处理缺失值的函数如下所示：</font>

|函数名|描述|
|:---:|:---:|
|dropna|根据每个标签的值是否是缺失数据来筛选轴标签，并允许根据丢失的数据量来确定阈值|
|fillna|用某些之填充缺失的数据或使用插值方法（如`ffill`或`bfill`）|
|isnull|返回表明哪些值是缺失值的布尔值|
|notnull|isnull的反函数|

In [29]:
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [30]:
# pandas遵循R语言的惯例，将缺失值称为NA(Not Available)。在统计学应用中，NA数据可以是不存在的数据或是存在但不可观察的数据（如在数据收集过程出现了问题）
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

### ①过滤缺失值（Filtering Out Missing Data）

In [31]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()       # 等价于：data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

In [32]:
# 在处理DataFrame对象时，dropna默认情况下会删除包含缺失值的行
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                     [NA, NA, NA], [NA, 6.5, 3.]])
cleaned = data.dropna()
print(data)
print('\ndropna默认：\n', cleaned)
# 传入how='all'参数后，会删除所有值均为NA的行
print('\ndata.dropna(how=\'all\')：\n', data.dropna(how='all'))
# 同样的方法删除列时：
data[4] = NA
data.dropna(axis=1, how='all')


     0    1    2
0  1.0  6.5  3.0
1  1.0  NaN  NaN
2  NaN  NaN  NaN
3  NaN  6.5  3.0

dropna默认：
      0    1    2
0  1.0  6.5  3.0

data.dropna(how='all')：
      0    1    2
0  1.0  6.5  3.0
1  1.0  NaN  NaN
3  NaN  6.5  3.0


,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [33]:
# 涉及到时间序列而想保留一定数量的观察值的行，用thresh参数来表示

df = pd.DataFrame(np.random.randn(7, 3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
print(df, '\n')
print(df.dropna(), '\n')
df.dropna(thresh=2)

          0         1         2
0  1.472825       NaN       NaN
1  0.428076       NaN       NaN
2 -0.774545       NaN -1.879406
3  0.786354       NaN -1.551229
4 -0.544796 -0.465915  1.164981
5 -0.623443 -0.143102 -0.121173
6  0.648957 -0.121546 -1.584526 

          0         1         2
4 -0.544796 -0.465915  1.164981
5 -0.623443 -0.143102 -0.121173
6  0.648957 -0.121546 -1.584526 



,0,1,2
2,-0.774545,NaN,-1.879406
3,0.786354,NaN,-1.551229
4,-0.544796,-0.465915,1.164981
5,-0.623443,-0.143102,-0.121173
6,0.648957,-0.121546,-1.584526


### ②补全缺失值（Filling In Missing Data）

<font color='red'>`fillna` 函数参数如下：</font>

|参数|描述|
|:---:|:---:|
|value|标量值或字典型对象用于填充缺失值|
|method|插值方法，如果没有其他参数，默认是 `ffill`|
|axis|需要填充的轴，默认是 `axis=0`|
|inplace|修改被调用的对象，而不是生成一个备份|
|limit|用于前向或后向填充时最大的填充范围|

In [34]:
print(df.fillna(0), '\n')
# 在调用时使用字典，可以为不同列的NA设定不同的填充值
print(df.fillna({1: 0.5, 2: 0}))

df.fillna(0, inplace=True)
df

          0         1         2
0  1.472825  0.000000  0.000000
1  0.428076  0.000000  0.000000
2 -0.774545  0.000000 -1.879406
3  0.786354  0.000000 -1.551229
4 -0.544796 -0.465915  1.164981
5 -0.623443 -0.143102 -0.121173
6  0.648957 -0.121546 -1.584526 

          0         1         2
0  1.472825  0.500000  0.000000
1  0.428076  0.500000  0.000000
2 -0.774545  0.500000 -1.879406
3  0.786354  0.500000 -1.551229
4 -0.544796 -0.465915  1.164981
5 -0.623443 -0.143102 -0.121173
6  0.648957 -0.121546 -1.584526


,0,1,2
0,1.472825,0.000000,0.000000
1,0.428076,0.000000,0.000000
2,-0.774545,0.000000,-1.879406
3,0.786354,0.000000,-1.551229
4,-0.544796,-0.465915,1.164981
5,-0.623443,-0.143102,-0.121173
6,0.648957,-0.121546,-1.584526


In [35]:
df = pd.DataFrame(np.random.randn(6, 3))
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
print(df)
# 可以将行列的平均值赋值给缺失值
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

          0         1         2
0 -0.170107  0.160044  0.347338
1  0.642997  1.480010  0.586302
2  1.001839       NaN -0.747983
3  0.460004       NaN -0.252441
4  1.514163       NaN       NaN
5 -0.476165       NaN       NaN


,0,1,2
0,-0.170107,0.160044,0.347338
1,0.642997,1.480010,0.586302
2,1.001839,1.480010,-0.747983
3,0.460004,1.480010,-0.252441
4,1.514163,NaN,-0.252441
5,-0.476165,NaN,-0.252441


## 2.数据转换（Data Transformation）

### ①删除重复值（Removing Duplicates）

`duplicated()`返回一个布尔值Series，反映每一行是否存在重复。

`drop_duplicates()`接收列标签作为参数并以此列为基础去除重复值。

两者都默认保存第一个观测到的值

In [36]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                     'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [37]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [38]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


In [39]:
data['v1'] = range(7)
data.drop_duplicates(['k1'])    # 以k1中的元素为基础对比重复值

,k1,k2,v1
0,one,1,0
1,two,1,1


### ②使用函数或映射进行数据转换（Transforming Data Using a Function or Mapping）

`map()`

In [40]:
# 关于肉类的假设数据
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                     'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [41]:
# 每种肉类来源的动物类型
meat_to_animal = {
  'bacon': 'pig',
  'pulled pork': 'pig',
  'pastrami': 'cow',
  'corned beef': 'cow',
  'honey ham': 'pig',
  'nova lox': 'salmon'
}

In [42]:
# 将上述两表合并
lowercased = data['food'].str.lower()       # 这里最好成为习惯，统一大小写，应用Series的方法
print(lowercased)                           # 或：data['food'].map(lambda x: meat_to_animal[x.lower()])
data['animal'] = lowercased.map(meat_to_animal)
data

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object


,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### ③替代值（Replacing Values）

`replace(inplace=True)`

In [43]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
print(data, '\n')
# -999可能是缺失值的标识，可以用replace方法生成新的Series
data.replace([-999, -1000], [np.nan, 0])
# 也可以通过字典来传递
data.replace({-999: np.nan, -1000: 0})

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64 



0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### ④重命名轴索引（Renaming Axis Indexes）

`index.map(func)` -> 对轴标签的函数或某种映射的原数据转换

`rename(index={}, columns={})` -> 生成一份拷贝数据，对其进行相应的行、列标签转换

In [44]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), 
                    index=['Ohio', 'Colorado', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [45]:
# 与Series的值一样，可以通过函数或某种形式的映射对轴标签进行类似的转换
transform = lambda x: x[:4].upper()
print(data.index.map(transform))

# 可以将修改后的行标签赋值给DataFrame
data.index = data.index.map(transform)
data

Index(['OHIO', 'COLO', 'NEW '], dtype='object')


,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [46]:
# 只创建新的数据集转换后的版本，而不修改原有的数据集
print(data.rename(index=str.title, columns=str.upper))

# 结合字典型对象使用，为轴标签的子集提供新的值
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'})

      ONE  TWO  THREE  FOUR
Ohio    0    1      2     3
Colo    4    5      6     7
New     8    9     10    11


,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


### ⑤离散化和分箱（iscretization and Binning）

`pd.cut(data, boundary_list, right=False, labels=group_names)`

`pd.cut(data, total_number, percision=int)`

`pd.qcut(data, boundary_list)`或`pd.qcut(data, total_number)`使用样本的分位数，可以获得等长的箱

- `pd.cut()`

In [47]:
# 假设现有某项研究中一组人群的年龄数据
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
# 将其分成18~25，26~35，36~60及60以上等若干组
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)   # 默认左开右闭（即右边能取到值），可以传入参数改变：pd.cut(ages, bins, right=False)
print(cats, '\n')
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
print('给分出来的每个箱命名，左闭右开：\n', pd.cut(ages, bins, right=False, labels=group_names))

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]] 

给分出来的每个箱命名，左闭右开：
 ['Youth', 'Youth', 'YoungAdult', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']


In [48]:
print(cats.codes, '\n')         # 不同年龄段区间的年龄用数字表示：0、1、2、3
# cut 计算出来的箱可以将其当作一个表示箱名的字符串数组，内部包含一个categories数组
print(cats.categories, '\n')    
pd.value_counts(cats)           # 对cut后的结果中的箱数量的统计

[0 0 0 1 0 0 2 1 3 2 2 1] 

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]') 



(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

In [49]:
# 传给cut整数以替代序列，表明要平均分成该整数份，那么pandas会根据最大最小值来平均分
data = np.random.rand(20)
pd.cut(data, 4, precision=2)        # precision将十进制精度限制在两位以内

[(0.27, 0.5], (0.048, 0.27], (0.048, 0.27], (0.048, 0.27], (0.27, 0.5], ..., (0.72, 0.95], (0.27, 0.5], (0.5, 0.72], (0.5, 0.72], (0.72, 0.95]]
Length: 20
Categories (4, interval[float64, right]): [(0.048, 0.27] < (0.27, 0.5] < (0.5, 0.72] < (0.72, 0.95]]

- `pd.qcut()`

In [50]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)      # Cut into quartiles 获得等长的箱（即每个箱中都有相同数量的数据点，这里是切成四份）
print(cats, '\n')
pd.value_counts(cats)

[(0.657, 3.551], (-3.056, -0.649], (0.657, 3.551], (0.0214, 0.657], (-3.056, -0.649], ..., (0.0214, 0.657], (0.0214, 0.657], (0.0214, 0.657], (-0.649, 0.0214], (0.0214, 0.657]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.056, -0.649] < (-0.649, 0.0214] < (0.0214, 0.657] < (0.657, 3.551]] 



(-3.056, -0.649]    250
(-0.649, 0.0214]    250
(0.0214, 0.657]     250
(0.657, 3.551]      250
dtype: int64

In [51]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(0.0214, 1.262], (-3.056, -1.272], (1.262, 3.551], (0.0214, 1.262], (-1.272, 0.0214], ..., (0.0214, 1.262], (0.0214, 1.262], (0.0214, 1.262], (-1.272, 0.0214], (0.0214, 1.262]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.056, -1.272] < (-1.272, 0.0214] < (0.0214, 1.262] < (1.262, 3.551]]

### ⑥检测和过滤异常值（Detecting and Filtering Outliers）


In [52]:
data = pd.DataFrame(np.random.randn(1000, 4))
print(data.describe(), '\n')

# 找出一列中绝对值大于三的值：
col = data[2]
print('找出一列(第三列)中绝对值大于三的值：\n', col[np.abs(col) > 3], '\n')

# 选出所有值大于3或小于-3的行
print('选出所有值大于3或小于-3的行：\n', data[(np.abs(data) > 3).any(1)], '\n')

                 0            1            2            3
count  1000.000000  1000.000000  1000.000000  1000.000000
mean     -0.002887     0.014316    -0.013293    -0.019726
std       1.014237     0.984397     1.016207     1.005068
min      -2.895441    -3.327861    -3.994854    -3.761469
25%      -0.646746    -0.705841    -0.690541    -0.691992
50%       0.011670     0.016161    -0.045462    -0.037424
75%       0.681804     0.666824     0.686338     0.646764
max       3.430827     3.448750     3.251199     3.512153 

找出一列(第三列)中绝对值大于三的值：
 288    3.021247
517    3.251199
669   -3.994854
873    3.039200
Name: 2, dtype: float64 

选出所有值大于3或小于-3的行：
             0         1         2         3
22   0.181072  0.994105 -0.305638 -3.104644
288 -1.233557 -2.314424  3.021247 -0.495756
354  0.304493  0.251225  0.880338  3.512153
398  0.633200  3.448750 -1.290489  1.589164
517 -0.651718  0.264682  3.251199 -0.520289
669 -0.144512  0.712130 -3.994854 -0.694218
680  3.430827 -0.644955 -0.007914  0.47

In [53]:
# 根据data中的值的正负分别生成1和-1的数值
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,1.0,-1.0
1,1.0,1.0,-1.0,-1.0
2,-1.0,-1.0,1.0,1.0
3,-1.0,1.0,-1.0,-1.0
4,-1.0,-1.0,1.0,-1.0


### ⑦置换和随机抽样（Permutation and Random Sampling）

`np,random.permutation`对DataFrame中的Series或行进行置换（随机重排序）

In [54]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5)
sampler

array([4, 1, 0, 3, 2])

In [55]:
print(df, '\n')
df.take(sampler)

    0   1   2   3
0   0   1   2   3
1   4   5   6   7
2   8   9  10  11
3  12  13  14  15
4  16  17  18  19 



,0,1,2,3
4,16,17,18,19
1,4,5,6,7
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11


In [56]:
# 选出一个不含有替代值的随机子集
print('选出一个不含有替代值的随机子集：\n', df.sample(n=3), '\n')

# 生成一个带有替代值的样本
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
print('生成一个带有替代值的样本：\n', draws, '\n')

选出一个不含有替代值的随机子集：
     0   1   2   3
2   8   9  10  11
4  16  17  18  19
0   0   1   2   3 

生成一个带有替代值的样本：
 3    6
1    7
0    5
1    7
1    7
0    5
3    6
4    4
3    6
2   -1
dtype: int64 



### ⑧计算指标/虚拟变量（Computing Indicator/Dummy Variables）

如果DataFrame中的一列有k个不同的值，则可以衍生出一个k列的值为1和0的矩阵或DataFrame。

`pd.get_dummies()`

In [57]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                   'data1': range(6)})
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [58]:
# 加入前缀，然后与其他数据合并
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


In [59]:
# 一行由多个类别时：
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                       header=None, names=mnames)
movies[:10]

/var/folders/v0/q2247qvj6nx2z3kxhc2_snyw0000gn/T/ipykernel_1365/3798072317.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movies.dat', sep='::',


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [60]:
# 为每个电影流派添加指标变量

# 首先，从数据集中提取所有不同的流派的列表：
all_genres = []
# 从DataFrame中取一行Series
for x in movies.genres:
    all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)              # 去重
print('提取出的电影流派：\n', genres, '\n')
# 使用全0的DataFrame
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

# 然后，遍历每一部电影，将dummies每一行的条目设置为1
gen = movies.genres[0]
print('gen.split(\'|\') -> ', gen.split('|'), '\n')
print('dummies.columns.get_indexer(gen.split(\'|\')) -> ', dummies.columns.get_indexer(gen.split('|')))
for i, gen in enumerate(movies.genres):
    indices = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indices] = 1

# 最后，将结果与movies联合
movies_windic = movies.join(dummies.add_prefix('Genre_'))
movies_windic.iloc[0]

提取出的电影流派：
 ['Animation' "Children's" 'Comedy' 'Adventure' 'Fantasy' 'Romance' 'Drama'
 'Action' 'Crime' 'Thriller' 'Horror' 'Sci-Fi' 'Documentary' 'War'
 'Musical' 'Mystery' 'Film-Noir' 'Western'] 

gen.split('|') ->  ['Animation', "Children's", 'Comedy'] 

dummies.columns.get_indexer(gen.split('|')) ->  [0 1 2]


movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                              1.0
Genre_Children's                             1.0
Genre_Comedy                                 1.0
Genre_Adventure                              0.0
Genre_Fantasy                                0.0
Genre_Romance                                0.0
Genre_Drama                                  0.0
Genre_Action                                 0.0
Genre_Crime                                  0.0
Genre_Thriller                               0.0
Genre_Horror                                 0.0
Genre_Sci-Fi                                 0.0
Genre_Documentary                            0.0
Genre_War                                    0.0
Genre_Musical                                0.0
Genre_Mystery                                0.0
Genre_Film-Noir                              0.0
Genre_Western       

In [61]:
# 写一个直接将NumPy数据写为NumPy数组的底层函数，将结果封装进DataFrame中
np.random.seed(12345)               # 用随机种子设定保持实例结果的确定性
values = np.random.rand(10)
print(values, '\n')
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

[0.92961609 0.31637555 0.18391881 0.20456028 0.56772503 0.5955447
 0.96451452 0.6531771  0.74890664 0.65356987] 



,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## 3.字符串操作（String Manipulation）

### ①字符串对象方法（String Object Methods）

`str.split(',')`

<font color='red'>Python内建的字符串方法：</font>

|方法|描述|
|:---:|:---:|
|count|返回子字符串在字符串中的非重叠出现次数|
|endswith|如果字符串以后缀结尾则返回True|
|startswith|如果字符串以前缀开始则返回True|
|join|使用字符串作为分隔字符，用于粘合其他字符串的序列|
|index|如果在字符串中找到，则返回字符串中第一个字符的位置，如果找不到就返回`ValueError`|
|find|返回字符串中第一个出现子字符的第一个字符的位置，如果没有找到，就返回-1|
|rfind|返回子字符串在字符串中最后一次出现时第一个字符的位置，如果没有找到，就返回-1|
|replace|使用一个字符串替代另一个字符串|
|strip、rstrip、lstrip|修剪空白、包括换行符；相当于对每个字符进行`x.strip()`|
|split|使用分隔符将字符串拆分成子字符串的列表|
|lower|将大写字母转换成小写字母|
|upper|将小写字符转换成大写字符|
|casefold|将字符转换为小写，并将任何位于特定区域的变量字符组合转换为常见的可比较形式|
|ljust、rjust|左对齐或右对齐，用空格或其他字符填充字符串的相反侧以返回具有最小宽度的字符串|

In [62]:
# 拆分多块
val = 'a,b,  guido'
val.split(',')

['a', 'b', '  guido']

In [63]:
# 加上去除空格
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [64]:
# 连接字符串
first, second, third = pieces
first + '::' + second + '::' + third        # 或：'::'.join(pieces)

'a::b::guido'

In [65]:
# 检测字符串
print('Is \'guido\' in val: ', 'guido' in val, '\n')
print(val.index(','), '\n')
val.find(':')
# index在字符串没找到时会抛出一个异常，而find则会返回-1

Is 'guido' in val:  True 

1 



-1

In [66]:
# 统计数量
val.count(',')

2

In [67]:
# 替代值
val.replace(',', '::')
val.replace(',', '')

'ab  guido'

### ②正则表达式（Regular Expressions）

单个表达式通常被称为 *regex*，避免转义字符影响，使用`r`或`C:\\x`

|方法|描述|
|:---:|:---:|
|findall|将字符串中所有的非重叠匹配模式以列表的形式返回|
|finditer|与findall类似，但返回的是迭代器|
|match|在字符串起始位置匹配模式，依然可以将模式组建匹配到分组中；如果模式匹配上了，返回一个匹配对象，否则返回None|
|search|扫描字符串的匹配模式，可以从任意位置开始扫描，返回第一个匹配项|
|split|根据模式，将字符串拆分成多个部分|
|sub, subn|用替换表达式替换字符串中所有匹配（sub）或第n个出现的匹配串（subn）；使用符号`\1, \2 ...`来引用替换字符串中的匹配组元素|

In [68]:
# 将含有多种空白字符（制表符、空格、换行符）的字符串拆分开
import re
text = "foo    bar\t baz  \tqux"    
re.split('\s+', text)       # 描述一个或多个空白字符的正则表达式：'\s+'


['foo', 'bar', 'baz', 'qux']

In [69]:
regex = re.compile('\s+')       # 自动编译，为保证及使用多个字符串同时编译处理时也能节省CPU的时间周期
print(regex.split(text))


['foo', 'bar', 'baz', 'qux']


In [70]:
text = """Dave dave@google.com
Steve steve@gmail.com
Rob rob@gmail.com
Ryan ryan@yahoo.com
"""
pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

# findall返回字符串中所有匹配项
print(regex.findall(text), '\n')

# search返回第一个匹配项
m = regex.search(text)
print(text[m.start():m.end()], '\n')

# regex.match只在模式出现于字符串起始位置进行匹配，如果没匹配到则返回None
print(regex.match(text), '\n')

print(regex.sub('REDACTED', text))

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com'] 

dave@google.com 

None 

Dave REDACTED
Steve REDACTED
Rob REDACTED
Ryan REDACTED



In [71]:
# 假设要查找用户名、域名、域名后缀名，并将其拆包
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,4})'
regex = re.compile(pattern, flags=re.IGNORECASE)

m = regex.match('wesm@bright.net')
print(m.groups(), '\n')

# 使用findall分组
print(regex.findall(text), '\n')

print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

('wesm', 'bright', 'net') 

[('dave', 'google', 'com'), ('steve', 'gmail', 'com'), ('rob', 'gmail', 'com'), ('ryan', 'yahoo', 'com')] 

Dave Username: dave, Domain: google, Suffix: com
Steve Username: steve, Domain: gmail, Suffix: com
Rob Username: rob, Domain: gmail, Suffix: com
Ryan Username: ryan, Domain: yahoo, Suffix: com



### ③pandas中的向量化字符串函数（Vectorized String Functions in pandas）

<font color='red'>部分向量化字符串方法列表</font>

|方法|描述|
|:--:|:--:|
|cat|根据可选的分隔符按元素黏合字符串|
|contains|返回是否含有某个模式\正则表达式|
|count|模式出现次数的统计|
|extract|使用正则表达式从字符串Series中分组抽取一个或多个字符串，返回结果是每个分组形成一列的DataFrame|
|endswith|等价于对每个元素使用 `x.endswith` 模式|
|startswith|等价于对每个元素使用 `x.endswith` 模式|
|findall|找出字符串中所有的模式\正则表达式匹配项，以列表返回|
|get|对每个元素进行索引（获得第i个元素）|
|isalnum|等价于内建的 `str.alnum`|
|isalpha|等价于内建的 `str.alpha`|
|isdecimal|等价于内建的 `str.isdecimal`|
|isdigit|等价于内建的 `str.isdigit`|
|islower|等价于内建的 `str.islower`|
|isnumeric|等价于内建的 `str.isnumeric`|
|isupper|等价于内建的 `str.isupper`|
|join|根据传递的分隔符，将Series中的字符串联合|
|len|计算每个字符串的长度|
|lower，upper|转换大小写，等价于对每个元素进行 `x.lower()` 或 `x.upper()`|
|match|使用 `re.match` 将正则表达式用于每个元素上，将匹配分组以列表形式返回|
|pad|将空白加到字符串的左边、右边、两边|
|center|等价于 `pad(side='both')`|
|repeat|重复值（例如 `s.str.repeat(3)` 等价于对每个字符串进行 x*3）|
|replace|以其他字符串替代模式，正式表达式的匹配项|
|slice|对Series中的字符串进行切片|
|strip|对字符串两侧的空白进行消除，包括换行符|
|rstrip|消除字符串右边的空白|
|lstrip|消除字符串左边的空白|

In [74]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
print(data, '\n')
print(data.isnull(), '\n')
# 检测是否包含域名
data.str.contains('gmail')

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object 

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool 



Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [75]:
pattern
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [80]:
# 使用str.get或str属性内部索引
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [82]:
# 访问嵌入式列表
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object